In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.colab import drive
from IPython.core.interactiveshell import InteractiveShell
import plotly.offline
import plotly.express as px
import gc
import warnings
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 100
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
os.chdir('/content/gdrive/MyDrive/msba6421_predictive/Amy_group_project/m5-forecasting-accuracy')

In [ ]:
!pwd

/content/gdrive/MyDrive/msba6421_predictive/Amy_group_project/m5-forecasting-accuracy


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Dec  2 23:04:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    28W /  70W |    309MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Fetch Data

In [ ]:
PROCESSED_DATA = '1202_data.pkl'

In [ ]:
sales = pd.read_csv('sales_train_evaluation.csv')
calendar = pd.read_csv('calendar.csv')
price = pd.read_csv('sell_prices.csv')

# Downcast Data

In [ ]:
#Downcast in order to save memory
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df


In [ ]:
sales = downcast(sales)
price = downcast(price)
calendar = downcast(calendar)

# Melting Data

In [ ]:
# create prediction date
for d in range(1942,1970):
    col = 'd_' + str(d)
    sales[col] = np.nan

In [ ]:
sales.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966',
       'd_1967', 'd_1968', 'd_1969'],
      dtype='object', length=1975)

In [ ]:
price['release'] = price.groupby(['store_id','item_id'])['wm_yr_wk'].transform('min')

In [ ]:
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold')

df = pd.merge(df, calendar[['d', 'wm_yr_wk']], on='d', how='left')
df = pd.merge(df, price, on=['store_id','item_id','wm_yr_wk'], how='left')

In [ ]:
df = df.loc[df['wm_yr_wk']>=df['release']]

In [ ]:
df['d'].unique()

array(['d_1', 'd_2', 'd_3', ..., 'd_1967', 'd_1968', 'd_1969'],
      dtype=object)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [ ]:
df = pd.merge(df, calendar.drop('wm_yr_wk', axis = 1), on='d', how='left')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47735397 entries, 0 to 47735396
Data columns (total 23 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            category      
 1   item_id       category      
 2   dept_id       category      
 3   cat_id        category      
 4   store_id      category      
 5   state_id      category      
 6   d             object        
 7   sold          float64       
 8   wm_yr_wk      int16         
 9   sell_price    float16       
 10  release       float64       
 11  date          datetime64[ns]
 12  weekday       category      
 13  wday          int8          
 14  month         int8          
 15  year          int16         
 16  event_name_1  category      
 17  event_type_1  category      
 18  event_name_2  category      
 19  event_type_2  category      
 20  snap_CA       int8          
 21  snap_TX       int8          
 22  snap_WI       int8          
dtypes: category(11), datetime64[ns

In [ ]:
del calendar, price, sales
gc.collect()

0

# EDA: Sales Data

## Random sample 10 items
1. overall low sales, no significant spikes
2. no obvious trends or seasonality, which could means a lot of noise

In [ ]:
random_items = df['id'].sample(10)
temp1 = df.loc[df['id'].isin(random_items)]
temp1['id'] = temp1['id'].astype(str)

In [ ]:
px.line(temp1, x = 'date', y = 'sold', color='id')

In [ ]:
del temp1
gc.collect()

419

## Overall
1. upward trend
2. drop in Christmas
3. weekly seasonality

In [ ]:
temp2 = df[['date','sold']].groupby('date').sum('sold').reset_index()
px.line(temp2, x = 'date', y = 'sold')

In [ ]:
del temp2
gc.collect()

310

## Overall by category

In [ ]:
temp3 = df[['date', 'cat_id','sold']].groupby(['date', 'cat_id']).sum('sold').reset_index()
temp3['cat_id'] = temp3['cat_id'].astype(str)
px.line(temp3, x = 'date', y = 'sold', color = 'cat_id')

In [ ]:
del temp3
gc.collect()

422

## Monthly Sales Per State

In [ ]:
temp4 =  df[['year', 'month', 'state_id','sold']]

In [ ]:
temp4 = temp4.groupby(['year', 'month', 'state_id'])['sold'].sum().reset_index()

In [ ]:
temp4['Date'] = pd.to_datetime(temp4[['year', 'month']].assign(DAY=1))

In [ ]:
temp4['state_id'] = temp4['state_id'].astype('str')

In [ ]:
px.line(temp4, x = ['Date'], y = 'sold'
, color = 'state_id')

In [ ]:
del temp4
gc.collect()

383

## Store Per State
1. sales patterns of same category across states varies a lot. Food in WI grow a lot
2. sales patterns of different categories within same state vary a lot.

In [ ]:
temp5 =  df[['year', 'month', 'state_id', 'cat_id', 'sold']]
temp5 = temp5.groupby(['year', 'month', 'state_id', 'cat_id'])['sold'].sum().reset_index()
temp5['date'] = pd.to_datetime(temp5[['year', 'month']].assign(DAY=1))
temp5['state_id'] = temp5['state_id'].astype('str')
temp5['cat_id'] = temp5['cat_id'].astype('str')

In [ ]:
px.line(temp5, x='date', y='sold', color='cat_id', facet_col='state_id',
              title='Category Sales Per State')

In [ ]:
del temp5
gc.collect()

0

## Weekdays Patterns
1. Weekends tend to have higher relative sales

In [ ]:
temp6 =  df[['weekday', 'state_id', 'sold']]
temp6 = temp6.groupby(['weekday', 'state_id'])['sold'].sum().reset_index()
temp6['state_id'] = temp6['state_id'].astype('str')

In [ ]:
temp6['relative_diff'] = temp6.groupby(['state_id'])['sold'].transform(lambda x: (x - x.mean()) / x.mean())


In [ ]:
index_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
px.imshow(
    temp6.pivot_table(index='state_id', columns='weekday', values='relative_diff')[index_order].values,
    x=index_order,
    y=temp6['state_id'].unique(),
    color_continuous_scale='RdYlBu_r',
    title='Relative Difference of Sales Across Weekdays and States'
)

In [ ]:
del temp6
gc.collect()

25

In [ ]:
temp7 =  df[['weekday', 'state_id', 'sold', 'month']]
temp7 = temp7.groupby(['weekday', 'state_id', 'month'])['sold'].sum().reset_index()
temp7['state_id'] = temp7['state_id'].astype('str')
temp7['month'] = temp7['month'].astype('str')

In [ ]:
temp7['relative_diff'] = temp7.groupby(['state_id','month'])['sold'].transform(lambda x: (x - x.mean()) / x.mean())

In [ ]:
index_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for index, s in enumerate( ['CA', 'TX', 'WI']):

    temp = temp7.loc[temp7['state_id'] == s]
    px.imshow(
        temp.pivot_table(index='month', columns='weekday', values='relative_diff')[index_order].values,
        x=index_order,
        y=temp7['month'].unique(),
        color_continuous_scale='RdYlBu_r',
        title=f'{s}: Relative Difference of Sales Across Weekdays and Months'
    )

In [ ]:
del temp7
gc.collect()

64

# Feature Engineering

## Label Encoding

In [ ]:
le = LabelEncoder()
cat_vars = ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
for cat in cat_vars:
  df[cat] = le.fit_transform(df[cat])

In [ ]:
df.d = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
df.drop('date',axis=1,inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47735397 entries, 0 to 47735396
Data columns (total 22 columns):
 #   Column        Dtype   
---  ------        -----   
 0   id            category
 1   item_id       category
 2   dept_id       category
 3   cat_id        category
 4   store_id      category
 5   state_id      category
 6   d             int16   
 7   sold          float64 
 8   wm_yr_wk      int16   
 9   sell_price    float16 
 10  release       float64 
 11  weekday       category
 12  wday          int8    
 13  month         int8    
 14  year          int16   
 15  event_name_1  int64   
 16  event_type_1  int64   
 17  event_name_2  int64   
 18  event_type_2  int64   
 19  snap_CA       int8    
 20  snap_TX       int8    
 21  snap_WI       int8    
dtypes: category(7), float16(1), float64(2), int16(3), int64(4), int8(5)
memory usage: 3.5 GB


In [ ]:
gc.collect()

705

## Lag Columns

In [ ]:
#Introduce lags
lags = np.arange(0, 15, 1)

for lag in lags:
    df['sold_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)

## Rolling Window Statistics

In [ ]:
for roll in [7, 14, 30, 60, 180]:
  df[f'rm_{roll}'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(roll).mean())
  df[f'std_{roll}'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(roll).std())
  df[f'diff_rm_{roll}'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x : x.diff().rolling(roll).mean())
  df[f'max_{roll}'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(roll).max())
  df = downcast(df)

## Exponential Rolling Average

In [ ]:
df['ema_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.ewm(span=4, adjust = False).mean()).astype(np.float16)

## Time Since Last Event

In [ ]:
df['snap_CA_days_since_last'] = np.nan

ca = df.loc[df['state_id'] == 'CA'][['d', 'snap_CA']]
ca['diff'] = ca['d'].where(ca['snap_CA'] == 1).ffill()
ca['diff'] = ca['d'] - ca['diff']

df['snap_CA_days_since_last'].loc[df['state_id'] == 'CA'] = ca['diff']

In [ ]:
df['snap_TX_days_since_last'] = np.nan

tx = df.loc[df['state_id'] == 'TX'][['d', 'snap_TX']]
tx['diff'] = tx['d'].where(tx['snap_TX'] == 1).ffill()
tx['diff'] = tx['d'] - tx['diff']

df['snap_TX_days_since_last'].loc[df['state_id'] == 'TX'] = tx['diff']

In [ ]:
df['snap_WI_days_since_last'] = np.nan

wi = df.loc[df['state_id'] == 'WI'][['d', 'snap_WI']]
wi['diff'] = wi['d'].where(wi['snap_WI'] == 1).ffill()
wi['diff'] = wi['d'] - wi['diff']

df['snap_WI_days_since_last'].loc[df['state_id'] == 'WI'] = wi['diff']

In [ ]:
df['snap_CA_days_since_last'].unique()
df['snap_TX_days_since_last'].unique()
df['snap_WI_days_since_last'].unique()

array([nan,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,
       12., 13., 14., 15., 16., 17., 18., 19., 20., 21.])

array([nan,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,
       12., 13., 14., 15., 16.])

array([nan,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,
       12., 13., 14., 15., 16., 17.])

In [ ]:
gc.collect()

17

## Price

In [ ]:
df['price_max'] = df.groupby(['store_id','item_id'])['sell_price'].transform('max')
df['price_min'] = df.groupby(['store_id','item_id'])['sell_price'].transform('min')
df['price_std'] = df.groupby(['store_id','item_id'])['sell_price'].transform('std')
df['price_mean'] = df.groupby(['store_id','item_id'])['sell_price'].transform('mean')
df['prev_sell_price'] = df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))

gc.collect()

df['price_norm'] = df['sell_price']/df['price_max']
df['price_momentum'] = df['sell_price']/df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
df['price_momentum_m'] = df['sell_price']/df.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
df['price_momentum_y'] = df['sell_price']/df.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')

0

## Mean Encoding

In [ ]:
df['item_sold_avg'] = df.groupby('item_id')['sold'].transform('mean').astype(np.float16)
df['state_sold_avg'] = df.groupby('state_id')['sold'].transform('mean').astype(np.float16)
df['store_sold_avg'] = df.groupby('store_id')['sold'].transform('mean').astype(np.float16)
df['cat_sold_avg'] = df.groupby('cat_id')['sold'].transform('mean').astype(np.float16)
df['dept_sold_avg'] = df.groupby('dept_id')['sold'].transform('mean').astype(np.float16)
df['cat_dept_sold_avg'] = df.groupby(['cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
df['store_item_sold_avg'] = df.groupby(['store_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['cat_item_sold_avg'] = df.groupby(['cat_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['dept_item_sold_avg'] = df.groupby(['dept_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['state_store_sold_avg'] = df.groupby(['state_id','store_id'])['sold'].transform('mean').astype(np.float16)
df['state_store_cat_sold_avg'] = df.groupby(['state_id','store_id','cat_id'])['sold'].transform('mean').astype(np.float16)
df['store_cat_dept_sold_avg'] = df.groupby(['store_id','cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
df['store_cat_sold_avg'] = df.groupby(['store_id','cat_id'])['sold'].transform('mean').astype(np.float16)
df['store_dept_sold_avg'] = df.groupby(['store_id','dept_id'])['sold'].transform('mean').astype(np.float16)
df['state_cat_sold_avg'] = df.groupby(['state_id','cat_id'])['sold'].transform('mean').astype(np.float16)
df['state_dept_sold_avg'] = df.groupby(['state_id','dept_id'])['sold'].transform('mean').astype(np.float16)

In [ ]:
df = df.drop('release', axis = 1)

## days since SNAP

In [ ]:
df['d'].loc[(df['state_id']=='CA') & (df['snap_CA'] == 1)].min()

4

In [ ]:
df['snap_CA_days_since_last'].loc[(df['state_id']=='CA') & (df['snap_CA_days_since_last'].isna())] = 4 - df['d'].loc[(df['state_id']=='CA') & (df['snap_CA_days_since_last'].isna())]

In [ ]:
df.loc[(df['state_id']=='CA')]

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,wm_yr_wk,sell_price,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sold_lag_0,sold_lag_1,sold_lag_2,sold_lag_3,sold_lag_4,sold_lag_5,sold_lag_6,sold_lag_7,sold_lag_8,sold_lag_9,sold_lag_10,sold_lag_11,sold_lag_12,sold_lag_13,sold_lag_14,rm_7,std_7,diff_rm_7,max_7,rm_14,std_14,diff_rm_14,max_14,rm_30,std_30,diff_rm_30,max_30,rm_60,std_60,diff_rm_60,max_60,rm_180,std_180,diff_rm_180,max_180,ema_sold,snap_CA_days_since_last,snap_TX_days_since_last,snap_WI_days_since_last,price_max,price_min,price_std,price_mean,prev_sell_price,price_norm,price_momentum,price_momentum_m,price_momentum_y,item_sold_avg,state_sold_avg,store_sold_avg,cat_sold_avg,dept_sold_avg,cat_dept_sold_avg,store_item_sold_avg,cat_item_sold_avg,dept_item_sold_avg,state_store_sold_avg,state_store_cat_sold_avg,store_cat_dept_sold_avg
0,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,1,12.0,11101,0.459961,Saturday,1,1,2011,30,4,4,2,0,0,0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000000,3.0,NaN,NaN,0.500000,0.419922,0.019775,0.476318,NaN,0.919922,NaN,0.965820,0.949707,4.683594,1.573242,1.635742,0.708984,0.865234,0.865234,7.285156,4.683594,4.683594,1.635742,1.003906,1.259766
1,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,1,2.0,11101,1.559570,Saturday,1,1,2011,30,4,4,2,0,0,0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,3.0,NaN,NaN,1.769531,1.559570,0.032715,1.764648,NaN,0.881348,NaN,0.883301,0.896973,0.849609,1.573242,1.635742,0.708984,0.865234,0.865234,1.178711,0.849609,0.849609,1.635742,1.003906,1.259766
2,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,1,0.0,11101,3.169922,Saturday,1,1,2011,30,4,4,2,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,3.0,NaN,NaN,3.169922,2.970703,0.046143,2.982422,NaN,1.000000,NaN,1.065430,1.043945,0.610352,1.573242,1.635742,0.708984,0.865234,0.865234,0.716797,0.610352,0.610352,1.635742,1.003906,1.259766
3,HOBBIES_1_012_CA_1_evaluation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,1,0.0,11101,5.980469,Saturday,1,1,2011,30,4,4,2,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,3.0,NaN,NaN,6.519531,5.980469,0.115845,6.468750,NaN,0.917480,NaN,0.921875,0.959473,0.381104,1.573242,1.635742,0.708984,0.865234,0.865234,0.392090,0.381104,0.381104,1.635742,1.003906,1.259766
4,HOBBIES_1_015_CA_1_evaluation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,1,4.0,11101,0.700195,Saturday,1,1,2011,30,4,4,2,0,0,0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,3.0,NaN,NaN,0.720215,0.680176,0.011345,0.707031,NaN,0.972168,NaN,0.988770,1.001953,4.417969,1.573242,1.635742,0.708984,0.865234,0.865234,6.015625,4.417969,4.417969,1.635742,1.003906,1.259766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47717098,FOODS_3_823_CA_4_evaluation,FOODS_3_823,FOODS_3,FOODS,CA_4,CA,1969,NaN,11621,2.980469,Sunday,2,6,2016,16,3,2,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.708496,9.0,NaN,NaN,2.980469,2.480469,0.163696,2.812500,2.980469,1.000000,1.0,1.030273,1.012695,0.846191,1.573242,0.898926,2.107422,2.619141,2.619141,0.548828,0.

In [ ]:
df['d'].loc[(df['state_id']=='TX') & (df['snap_TX'] == 1)].min()

4

In [ ]:
df['snap_TX_days_since_last'].loc[(df['state_id']=='TX') & (df['snap_TX_days_since_last'].isna())] = 4 - df['d'].loc[(df['state_id']=='TX') & (df['snap_TX_days_since_last'].isna())]

In [ ]:
df['d'].loc[(df['state_id']=='WI') & (df['snap_WI'] == 1)].min()

5

In [ ]:
df['snap_WI_days_since_last'].loc[(df['state_id']=='WI') & (df['snap_WI_days_since_last'].isna())] = 5 - df['d'].loc[(df['state_id']=='WI') & (df['snap_WI_days_since_last'].isna())]

In [ ]:
df['snap_days_since_last'] = df[['snap_CA_days_since_last','snap_TX_days_since_last','snap_WI_days_since_last']].mean(axis=1, skipna=True)

In [ ]:
df[df['snap_days_since_last'].isna()]

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,wm_yr_wk,sell_price,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sold_lag_0,sold_lag_1,sold_lag_2,sold_lag_3,sold_lag_4,sold_lag_5,sold_lag_6,sold_lag_7,sold_lag_8,sold_lag_9,sold_lag_10,sold_lag_11,sold_lag_12,sold_lag_13,sold_lag_14,rm_7,std_7,diff_rm_7,max_7,rm_14,std_14,diff_rm_14,max_14,rm_30,std_30,diff_rm_30,max_30,rm_60,std_60,diff_rm_60,max_60,rm_180,std_180,diff_rm_180,max_180,ema_sold,snap_CA_days_since_last,snap_TX_days_since_last,snap_WI_days_since_last,price_max,price_min,price_std,price_mean,prev_sell_price,price_norm,price_momentum,price_momentum_m,price_momentum_y,item_sold_avg,state_sold_avg,store_sold_avg,cat_sold_avg,dept_sold_avg,cat_dept_sold_avg,store_item_sold_avg,cat_item_sold_avg,dept_item_sold_avg,state_store_sold_avg,state_store_cat_sold_avg,store_cat_dept_sold_avg,snap_days_since_last


In [ ]:
df = df.drop(['snap_CA_days_since_last','snap_TX_days_since_last','snap_WI_days_since_last'], axis = 1)

In [ ]:
df = downcast(df)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47735397 entries, 0 to 47735396
Data columns (total 83 columns):
 #   Column                    Dtype   
---  ------                    -----   
 0   id                        category
 1   item_id                   category
 2   dept_id                   category
 3   cat_id                    category
 4   store_id                  category
 5   state_id                  category
 6   d                         int16   
 7   sold                      float16 
 8   wm_yr_wk                  int16   
 9   sell_price                float16 
 10  weekday                   category
 11  wday                      int8    
 12  month                     int8    
 13  year                      int16   
 14  event_name_1              int8    
 15  event_type_1              int8    
 16  event_name_2              int8    
 17  event_type_2              int8    
 18  snap_CA                   int8    
 19  snap_TX                   int8    
 20  

In [ ]:
df.to_pickle(PROCESSED_DATA)